### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "PACS"
MODEL_NAME = "resnet18"

Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu126 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1029 12:00:21.972000 22444 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Device: cuda
PyTorch: 2.8.0+cu126


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), "PACS_MISC")
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-29 12:00:24,437 | INFO | Initialized experiment directories for PACS
2025-10-29 12:00:24,438 | INFO | Logs: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC\logs
2025-10-29 12:00:24,439 | INFO | Checkpoints: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC\checkpoints
2025-10-29 12:00:24,440 | INFO | Plots: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC\plots


### Setup

In [9]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints_Misc")
log_dir = os.path.join(base_dir, "logs_Misc")
plots_dir = os.path.join(base_dir, "plots_Misc")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
CFG["train"]["lr"] = 1e-4
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.0))
CFG["train"]["epochs"] = 10
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [10]:
CFG["grqo"]["num_tokens"] = 32
CFG["grqo"]["random_k"] = 32
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-29 12:29:28,237 | INFO | === LODO: Leaving out domain 'art_painting' ===



=== LODO: Leaving out domain 'art_painting' ===


Evaluating: 100%|██████████| 16/16 [00:03<00:00,  4.11it/s]
2025-10-29 12:29:46,088 | INFO | [art_painting] Epoch 1/10 | Train - Loss: 0.5734, Cls: 0.5648, GRQO: 0.0086, Acc: 0.8158 | Val - Loss: 0.7457, Cls: 0.7444, GRQO: 0.0012, Acc: 0.7705
2025-10-29 12:29:46,155 | INFO | [art_painting] New best val acc: 0.7705


[art_painting] Epoch 1/10 | Train - Loss: 0.5734, Cls: 0.5648, GRQO: 0.0086, Acc: 0.8158 | Val - Loss: 0.7457, Cls: 0.7444, GRQO: 0.0012, Acc: 0.7705
[art_painting] New best val acc: 0.7705


Evaluating: 100%|██████████| 16/16 [00:03<00:00,  4.17it/s]
2025-10-29 12:30:04,405 | INFO | [art_painting] Epoch 2/10 | Train - Loss: 0.0944, Cls: 0.0916, GRQO: 0.0028, Acc: 0.9669 | Val - Loss: 1.0490, Cls: 1.0481, GRQO: 0.0009, Acc: 0.7183


...

[sketch] Epoch 8/10 | Train - Loss: 0.0036, Cls: 0.0023, GRQO: 0.0013, Acc: 0.9995 | Val - Loss: 1.3636, Cls: 1.3635, GRQO: 0.0001, Acc: 0.7093


Evaluating: 100%|██████████| 31/31 [00:05<00:00,  5.35it/s]
2025-10-29 12:42:19,446 | INFO | [sketch] Epoch 9/10 | Train - Loss: 0.0044, Cls: 0.0032, GRQO: 0.0012, Acc: 0.9995 | Val - Loss: 1.6408, Cls: 1.6407, GRQO: 0.0001, Acc: 0.6717


[sketch] Epoch 9/10 | Train - Loss: 0.0044, Cls: 0.0032, GRQO: 0.0012, Acc: 0.9995 | Val - Loss: 1.6408, Cls: 1.6407, GRQO: 0.0001, Acc: 0.6717


Evaluating: 100%|██████████| 31/31 [00:05<00:00,  5.38it/s]
2025-10-29 12:42:38,681 | INFO | [sketch] Epoch 10/10 | Train - Loss: 0.0159, Cls: 0.0147, GRQO: 0.0012, Acc: 0.9964 | Val - Loss: 1.7374, Cls: 1.7372, GRQO: 0.0002, Acc: 0.6559
2025-10-29 12:42:38,681 | INFO | [sketch] Best Acc: 0.7216
2025-10-29 12:42:38,681 | INFO | ------------------------------------------------------------
2025-10-29 12:42:38,699 | INFO | LODO finished | Mean Acc: 0.7995 | Summary saved to d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC\logs_Misc\lodo_summary_20251029_124238.json


[sketch] Epoch 10/10 | Train - Loss: 0.0159, Cls: 0.0147, GRQO: 0.0012, Acc: 0.9964 | Val - Loss: 1.7374, Cls: 1.7372, GRQO: 0.0002, Acc: 0.6559
[sketch] Best Acc: 0.7216
------------------------------------------------------------
LODO finished | Mean Acc: 0.7995
Summary saved to d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18\PACS_MISC\logs_Misc\lodo_summary_20251029_124238.json


### Baseline

In [11]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-10-29 12:52:56,914 | INFO | Initializing ResNet baseline: resnet18
2025-10-29 12:52:57,025 | INFO | === Baseline LODO: Leaving out domain 'art_painting' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'art_painting' ===


2025-10-29 12:53:12,489 | INFO | [art_painting] Epoch 1/10 | Train - Loss: 0.4830, Acc: 0.8492 | Val Acc: 0.7822


[art_painting] Epoch 1/10 | Train - Loss: 0.4830, Acc: 0.8492 | Val Acc: 0.7822


2025-10-29 12:53:28,184 | INFO | [art_painting] Epoch 2/10 | Train - Loss: 0.0996, Acc: 0.9751 | Val Acc: 0.7300


...

[sketch] Epoch 8/10 | Train - Loss: 0.0020, Acc: 1.0000 | Val Acc: 0.6353


2025-10-29 13:03:37,074 | INFO | [sketch] Epoch 9/10 | Train - Loss: 0.0014, Acc: 1.0000 | Val Acc: 0.6526


[sketch] Epoch 9/10 | Train - Loss: 0.0014, Acc: 1.0000 | Val Acc: 0.6526


2025-10-29 13:03:56,056 | INFO | [sketch] Epoch 10/10 | Train - Loss: 0.0012, Acc: 1.0000 | Val Acc: 0.6559
2025-10-29 13:03:56,056 | INFO | [sketch] Best Val Acc: 0.6852
2025-10-29 13:03:56,056 | INFO | ------------------------------------------------------------
2025-10-29 13:03:56,056 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.7866


[sketch] Epoch 10/10 | Train - Loss: 0.0012, Acc: 1.0000 | Val Acc: 0.6559
[sketch] Best Val Acc: 0.6852
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.7866
